# Setup

In [10]:
import torch
import json
import os
import sys
import re

import pandas as pd

from fedot_llm.language_models.llms import HuggingFaceLLM
from fedot_llm.language_models.actions import ModelAction
from fedot_llm.data.data import Dataset
from fedot_llm.fedot_util import run_example
from pprint import pprint

# Загрузка данных

In [2]:
dataset_name = [
    'titanic', 
    'credit-g'
][0]
dataset_path = os.sep.join(['..', 'datasets', dataset_name])
dataset = Dataset.load_from_path(dataset_path)
dataset_description = dataset.get_description()
dataset_metadata_description = dataset.get_metadata_description()

print(dataset_description)
print()
print(dataset_metadata_description)

Assume we have a dataset
The dataset contains the following splits:
 
The test_merged split stored in file "test_merged.csv" contains following columns: ['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked', 'Survived']. It is described as None
The test split stored in file "test.csv" contains following columns: ['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']. It is described as None
The predictions split stored in file "predictions.csv" contains following columns: ['Unnamed: 0', 'Survived']. It is described as None
The train split stored in file "train.csv" contains following columns: ['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']. It is described as None
The gender_submission split stored in file "gender_submission.csv" contains following columns: ['PassengerId', 'Survived']. It is described as None

name: Non

In [3]:
import json
with open('../datasets/big_descriptions.json', 'r') as json_file:
    dataset_big_descriptions = json.load(json_file)
dataset_big_descriptions

{'titanic': 'The sinking of the Titanic is one of the most infamous shipwrecks in history.\n\nOn April 15, 1912, during her maiden voyage, the widely considered “unsinkable” RMS Titanic sank after colliding with an iceberg. Unfortunately, there weren’t enough lifeboats for everyone onboard, resulting in the death of 1502 out of 2224 passengers and crew.\n\nWhile there was some element of luck involved in surviving, it seems some groups of people were more likely to survive than others.\n\nIn this challenge, we ask you to build a predictive model that answers the question: “what sorts of people were more likely to survive?” using passenger data (ie name, age, gender, socio-economic class, etc).\n\nIn this competition, you’ll gain access to two similar datasets that include passenger information like name, age, gender, socio-economic class, etc. One dataset is titled train.csv and the other is titled test.csv.\n\nTrain.csv will contain the details of a subset of the passengers on board (

In [4]:
model = HuggingFaceLLM(model_id="microsoft/Phi-3-mini-4k-instruct", max_new_tokens=500)
action = ModelAction(model)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

# Уточнение данных о датасете

In [5]:
from fedot_llm.language_models import prompts

action = ModelAction(model=model)
# 1: Название датасета и определение тренировочного сплита
task_prompts = {
    "dataset_name": {
        "system": dataset_big_descriptions[dataset_name],
        "task": prompts.dataset_name_prompt,
        "context": None,
    },
    "train_split": {
        "system": dataset_big_descriptions[dataset_name],
        "task": prompts.train_split_definition_prompt,
        "context": dataset.get_description(),
    }
}

responses = action.run_model_multicall(
    task_prompts
)
operations = {
    "train_split": lambda x : x.split(".")[0]
}
responses = action.process_model_responses(responses, operations)
pprint(responses)

dataset.name = responses["dataset_name"]
dataset.train_split_name = responses["train_split"] 

The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
You are not running the flash-attention implementation, expect numerical differences.


{'dataset_name': 'Titanic Survival Prediction', 'train_split': 'train'}


In [6]:
train = list(filter(lambda split: split.name == dataset.train_split_name, dataset.splits))[0]
column_descriptions = action.generate_all_column_description(split=train, dataset=dataset)
train.set_column_descriptions(column_descriptions)
column_descriptions

{'PassengerId': 'the passengerid column represents the unique identifier assigned to each passenger on the titanic, ranging from 1 to 30.',
 'Survived': "the 'survived' column indicates whether a passenger survived (1) or did not survive (0) the titanic disaster.",
 'Pclass': "the 'pclass' column represents the passenger class on the titanic, with values indicating first class (1), second class (2), and third class (3).",
 'Name': "the 'name' column in the titanic survival prediction dataset contains the full names of passengers, including their titles and last names, which may provide insights into social status and potential survival rates.",
 'Sex': "the 'sex' column in the titanic dataset indicates the gender of passengers, with 'male' and 'female' as the possible values.",
 'Age': "the 'age' column in the titanic survival prediction dataset represents the age of passengers on the titanic, ranging from infants (0.42 years old) to elderly individuals (70.5 years old). the ages are r

In [9]:
# 2: Категориальные столбцы, таргет-столбец, тип задачи

task_prompts = {
    "categorical_columns": {
        "system": dataset_description,
        "task": prompts.categorical_definition_prompt,
        "context": prompts.categorical_definition_context,
    },
    "target_column": {
        "system": dataset_description,
        "task": prompts.target_definition_prompt,
        "context": None,
    },
    "task_type": {
        "system": dataset_description,
        "task": prompts.task_definition_prompt,
        "context": None,
    }
}

responses = action.run_model_multicall(
    task_prompts
)

pattern = r'[\'\"“”‘’`´]'
operations = {
    "categorical_columns": lambda x : x.split("\n"),
    "target_column" :  lambda x : re.sub(pattern, '', x),
    "task_type": lambda x : re.sub(pattern, '', x.lower())
}
responses = action.process_model_responses(responses, operations)
pprint(responses)

{'categorical_columns': ['Pclass',
                         'Sex',
                         'Age',
                         'SibSp',
                         'Parch',
                         'Fare',
                         'Cabin',
                         'Embarked'],
 'target_column': 'Survived',
 'task_type': 'classification'}


In [11]:
prediction = run_example(train_df=dataset.splits[3].data, test_df=dataset.splits[0].data, problem=responses['task_type'], target=responses['target_column'])

2024-07-24 17:48:31,088 - Topological features operation requires extra dependencies for time series forecasting, which are not installed. It can infuence the performance. Please install it by 'pip install fedot[extra]'


Generations:   0%|          | 0/10000 [00:00<?, ?gen/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_

2024-07-24 17:48:42,733 - Topological features operation requires extra dependencies for time series forecasting, which are not installed. It can infuence the performance. Please install it by 'pip install fedot[extra]'
2024-07-24 17:48:42,733 - Topological features operation requires extra dependencies for time series forecasting, which are not installed. It can infuence the performance. Please install it by 'pip install fedot[extra]'
2024-07-24 17:48:42,733 - Topological features operation requires extra dependencies for time series forecasting, which are not installed. It can infuence the performance. Please install it by 'pip install fedot[extra]'
2024-07-24 17:49:05,287 - Topological features operation requires extra dependencies for time series forecasting, which are not installed. It can infuence the performance. Please install it by 'pip install fedot[extra]'
2024-07-24 17:49:05,292 - Topological features operation requires extra dependencies for time series forecasting, which 

Generations:   0%|          | 0/10000 [18:16<?, ?gen/s]


{'roc_auc': 0.941, 'accuracy': 0.828}
